## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Prince Rupert,CA,54.3161,-130.3201,53.69,100,100,3.44,moderate rain
1,1,Puerto Ayora,EC,-0.7393,-90.3518,69.55,95,99,8.99,overcast clouds
2,2,Luderitz,NaN,-26.6481,15.1594,69.24,27,0,8.77,clear sky
3,3,Bluff,NZ,-46.6000,168.3333,47.26,75,95,22.93,overcast clouds
4,4,Isla Vista,US,34.4133,-119.8610,92.37,29,1,4.14,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input("What is the minimum temperature you'd like for your vacation? "))
max_temp = int(input("What is the maximum temperature you'd like for your vacation? "))

What is the minimum temperature you'd like for your vacation?55
What is the maximum temperature you'd like for your vacation?69


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_temp_range = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
city_temp_range.isnull().sum()

Unnamed: 0             0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_city_temp_range = city_temp_range.dropna()

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_city_temp_range[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Alghero,IT,66.29,clear sky,40.5589,8.3181,
9,Yellowknife,CA,64.09,broken clouds,62.4560,-114.3525,
20,Castro,BR,61.21,overcast clouds,-24.7911,-50.0119,
21,Jamestown,US,63.99,overcast clouds,42.0970,-79.2353,
26,Busselton,AU,59.36,overcast clouds,-33.6500,115.3333,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [47]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.loc[hotel_df["Hotel Name"] == ''] = np.nan
#hotel_df.isnull().sum()
#clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Alghero,IT,66.29,clear sky,40.5589,8.3181,Smy Carlos V Alghero
9,Yellowknife,CA,64.09,broken clouds,62.4560,-114.3525,The Explorer Hotel
20,Castro,BR,61.21,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
21,Jamestown,US,63.99,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
26,Busselton,AU,59.36,overcast clouds,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
669,Burns Lake,CA,58.42,broken clouds,54.2331,-125.7533,Burns Lake Bed and Breakfast
687,Antalaha,MG,67.77,light rain,-14.9003,50.2788,Hotel Vitasoa
692,Rovaniemi,FI,56.61,light intensity shower rain,66.5000,25.7167,Arctic City Hotel
705,Ahipara,NZ,55.45,few clouds,-35.1667,173.1667,Ahipara Holiday Park


In [48]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))